In [ ]:
pip install google-cloud-secret-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
from google.cloud import secretmanager

from google.auth import default

# Get default credentials and project id
credentials, project_id = default()


# Initialize the Secret Manager client
client = secretmanager.SecretManagerServiceClient()

# Define secret name
secret_id = "kavia-sec-API-key"  # the secret name you created

# Build the resource name of the secret version (latest)
name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"

# Access the secret version
response = client.access_secret_version(name=name)

# Get the secret payload as string
secret_value = response.payload.data.decode("UTF-8")

In [ ]:
import requests
import json
import time

url = "https://api.sec-api.io/sec-enforcement-actions?token=" + secret_value
headers = {
    "Content-Type": "application/json"
}

from_index = 0
total_rows = 11000

with open('/sec_actions_enforcement.json', 'w') as sec_actions_file:
    number_of_data = 0
    retry_wait = 1
    while from_index < total_rows :
        data = {
            "query": "releasedAt:[1997-01-01 TO 2025-09-06]",
            "from": from_index,
            "size": 50,
            "sort": [{"releasedAt": {"order": "desc"}}]
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 429:
            print(f"Rate limit exceeded, retrying in {retry_wait} seconds...")
            time.sleep(retry_wait)
            retry_wait = min(retry_wait * 2, 60)  # exponential backoff max 60s
            continue
        else:
            retry_wait = 1  # reset on success

        if response.status_code == 200:
            response_json = response.json()
            if total_rows != response_json.get('total').get('value'):
                total_rows = response_json.get('total').get('value')
                print(f"total rows changed to {total_rows}")
            response_list = response_json.get('data')
            for data_element in response_list:
                # Print each element as compact JSON (no spaces)
                print(json.dumps(data_element, separators=(',', ':'), ensure_ascii=False), file=sec_actions_file)
                number_of_data += 1
            from_index += 50
        else:
            print(f"Request failed with status code: {response.status_code}")
            print(response.text)
            break
    print(f"{number_of_data} inserted as separate lines in the file")

total rows changed to 2760
Rate limit exceeded, retrying in 1 seconds...
Rate limit exceeded, retrying in 1 seconds...
2760 inserted as separate lines in the file


In [ ]:
import json

with open('/batch0.json', 'r') as f:
    data = json.load(f)  # Load JSON array

data = data['data']

# f = open('/output1.json','a')

with open('/output1.json', 'w') as f:
    for element in data:
        # Print each element as compact JSON (no spaces)
        print(json.dumps(element, separators=(',', ':'), ensure_ascii=False), file=f)

In [ ]:
import requests

url = "https://api.sec-api.io/sec-enforcement-actions?token=" + secret_value
headers = {
    "Content-Type": "application/json"
}

data = {
    "query": "releasedAt:[1997-01-01 TO 2025-09-05]",
    "from": 0,
    "size": 50,
    "sort": [{"releasedAt": {"order": "desc"}}]
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    response_data = response.text

else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

In [ ]:
parsed_json = json.loads(response_data) # Step 2: parse JSON

total = parsed_json.get("total")   # Step 3: access total
data = parsed_json.get("data")     # Step 3: access data

print("Total:", total)
print("Data:", data)


Total: {'value': 2760, 'relation': 'eq'}
Data: [{'id': '5906a4d0a7fbda6ac2758f526f0d81e8', 'releaseNo': '2025-111', 'releasedAt': '2025-09-03T15:20:35Z', 'url': 'https://www.sec.gov/newsroom/press-releases/2025-111-sec-charges-pennsylvania-resident-his-companies-770-million-ponzi-scheme', 'title': 'SEC Charges Pennsylvania Resident and His Companies with $770 Million Ponzi Scheme', 'resources': [{'label': 'SEC Complaint', 'url': 'https://www.sec.gov/litigation/complaints/2025/comp-pr2025-111.pdf'}], 'summary': 'The SEC has charged Daryl F. Heller and his companies with operating a $770 million Ponzi scheme that resulted in $400 million in investor losses.', 'tags': ['ponzi scheme', 'disclosure fraud'], 'entities': [{'name': 'Daryl F. Heller', 'type': 'individual', 'role': 'defendant'}, {'name': 'Prestige Investment Group, LLC', 'type': 'company', 'role': 'defendant'}, {'name': 'Paramount Management Group, LLC', 'type': 'company', 'role': 'defendant'}], 'complaints': ['Heller and Presti

In [ ]:
response_data

'{"total":{"value":2760,"relation":"eq"},"data":[{"id":"5906a4d0a7fbda6ac2758f526f0d81e8","releaseNo":"2025-111","releasedAt":"2025-09-03T15:20:35Z","url":"https://www.sec.gov/newsroom/press-releases/2025-111-sec-charges-pennsylvania-resident-his-companies-770-million-ponzi-scheme","title":"SEC Charges Pennsylvania Resident and His Companies with $770 Million Ponzi Scheme","resources":[{"label":"SEC Complaint","url":"https://www.sec.gov/litigation/complaints/2025/comp-pr2025-111.pdf"}],"summary":"The SEC has charged Daryl F. Heller and his companies with operating a $770 million Ponzi scheme that resulted in $400 million in investor losses.","tags":["ponzi scheme","disclosure fraud"],"entities":[{"name":"Daryl F. Heller","type":"individual","role":"defendant"},{"name":"Prestige Investment Group, LLC","type":"company","role":"defendant"},{"name":"Paramount Management Group, LLC","type":"company","role":"defendant"}],"complaints":["Heller and Prestige raised more than $770 million from a

In [ ]:
response_data

{'total': {'value': 2760, 'relation': 'eq'},
 'data': [{'id': '5906a4d0a7fbda6ac2758f526f0d81e8',
   'releaseNo': '2025-111',
   'releasedAt': '2025-09-03T15:20:35Z',
   'url': 'https://www.sec.gov/newsroom/press-releases/2025-111-sec-charges-pennsylvania-resident-his-companies-770-million-ponzi-scheme',
   'title': 'SEC Charges Pennsylvania Resident and His Companies with $770 Million Ponzi Scheme',
   'resources': [{'label': 'SEC Complaint',
     'url': 'https://www.sec.gov/litigation/complaints/2025/comp-pr2025-111.pdf'}],
   'summary': 'The SEC has charged Daryl F. Heller and his companies with operating a $770 million Ponzi scheme that resulted in $400 million in investor losses.',
   'tags': ['ponzi scheme', 'disclosure fraud'],
   'entities': [{'name': 'Daryl F. Heller',
     'type': 'individual',
     'role': 'defendant'},
    {'name': 'Prestige Investment Group, LLC',
     'type': 'company',
     'role': 'defendant'},
    {'name': 'Paramount Management Group, LLC',
     'type'

In [ ]:
type(response_data)

dict

In [ ]:
type(response_data.get('data'))

list

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

source_table_id = "annular-surf-470214-a2.first_dataset.temp_sec_json"
destination_table_id = "financial-fraud-detection-1.sec_enforcement_actions.sec_actions"

job_config = bigquery.CopyJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

job = client.copy_table(
    sources = source_table_id,
    destination = destination_table_id,
    # Optional: write_disposition can be WRITE_TRUNCATE to overwrite
    job_config = job_config
)

job.result()  # Wait for the job to complete
print(f"Copied table {source_table_id} to {destination_table_id}")

Copied table annular-surf-470214-a2.first_dataset.temp_sec_json to financial-fraud-detection-1.sec_enforcement_actions.sec_actions


In [ ]:
import os
from google.cloud import bigquery

# After uploading the file via UI, give the path to the file on notebook VM
json_file_path = "/sec_actions_enforcement.json"  # example path in the notebook VM

client = bigquery.Client()  # Uses project associated with the notebook

dataset_id = "sec_enforcement_actions"
table_id = "sec_actions"

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE_DATA,
)

with open(json_file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    job.result()

print(f"Loaded {job.output_rows} rows into {table_id}")

BadRequest: 400 Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 2122; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 2122; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON processing encountered too many errors, giving up. Rows: 2122; errors: 1; max bad: 0; error percent: 0; reason: invalidQuery, location: query, message: Could not parse '2011-12-29' as a timestamp. Required format is YYYY-MM-DD HH:MM[:SS[.SSSSSS]] or YYYY/MM/DD HH:MM[:SS[.SSSSSS]] Field: releasedAt; Value: 2011-12-29

[Reference link](https://www.timeanddate.com/time/change/usa/washington-dc?year=1997)

In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Original DST dates (official transition dates at 2:00 AM)
dst_original = [
    # 1997-2006: First Sunday in April to Last Sunday in October
    {"Year": 1997, "Official_Start": "April 6", "Official_End": "October 26"},
    {"Year": 1998, "Official_Start": "April 5", "Official_End": "October 25"},
    {"Year": 1999, "Official_Start": "April 4", "Official_End": "October 31"},
    {"Year": 2000, "Official_Start": "April 2", "Official_End": "October 29"},
    {"Year": 2001, "Official_Start": "April 1", "Official_End": "October 28"},
    {"Year": 2002, "Official_Start": "April 7", "Official_End": "October 27"},
    {"Year": 2003, "Official_Start": "April 6", "Official_End": "October 26"},
    {"Year": 2004, "Official_Start": "April 4", "Official_End": "October 31"},
    {"Year": 2005, "Official_Start": "April 3", "Official_End": "October 30"},
    {"Year": 2006, "Official_Start": "April 2", "Official_End": "October 29"},
    # 2007-2012: Second Sunday in March to First Sunday in November
    {"Year": 2007, "Official_Start": "March 11", "Official_End": "November 4"},
    {"Year": 2008, "Official_Start": "March 9", "Official_End": "November 2"},
    {"Year": 2009, "Official_Start": "March 8", "Official_End": "November 1"},
    {"Year": 2010, "Official_Start": "March 14", "Official_End": "November 7"},
    {"Year": 2011, "Official_Start": "March 13", "Official_End": "November 6"},
    {"Year": 2012, "Official_Start": "March 11", "Official_End": "November 4"}
]

def parse_date_string(date_str, year):
    """Convert date string like 'April 6' to datetime object"""
    return datetime.strptime(f"{date_str} {year}", "%B %d %Y")

def add_one_day(date_str, year):
    """Add one day to the given date"""
    original_date = parse_date_string(date_str, year)
    next_day = original_date + timedelta(days=1)
    return next_day.strftime("%B %d")

# Create adjusted dates for 00:00:00 calculations
adjusted_data = []

for row in dst_original:
    year = row["Year"]

    # For DST START: Add 1 day because DST activates at 2:00 AM
    # So if you're checking at 00:00:00, you need the next day to be in DST
    adjusted_start = add_one_day(row["Official_Start"], year)

    # For DST END: Keep the same date because DST ends at 2:00 AM
    # So at 00:00:00 on the official end date, you're still in DST
    adjusted_end = row["Official_End"]

    adjusted_data.append({
        "Year": year,
        "Official_DST_Start": row["Official_Start"],
        "Adjusted_DST_Start_00h00": adjusted_start,
        "Official_DST_End": row["Official_End"],
        "Adjusted_DST_End_00h00": adjusted_end,
        "UTC_Offset_Change": "UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
    })

# Create DataFrame
df = pd.DataFrame(adjusted_data)

print("DST DATES ADJUSTED FOR 00:00:00 TIMESTAMP CALCULATIONS")
print("=" * 80)
print("LOGIC:")
print("- Official DST starts at 2:00 AM, so 00:00:00 on that day is still Standard Time")
print("- For 00:00:00 calculations, use the NEXT DAY as your DST start date")
print("- DST ends at 2:00 AM, so 00:00:00 on end day is still Daylight Time")
print("- For 00:00:00 calculations, use the SAME DAY as your DST end date")
print("=" * 80)
print()

# Display comparison
for _, row in df.iterrows():
    print(f"{row['Year']}:")
    print(f"  Official DST Start: {row['Official_DST_Start']} (changes at 2:00 AM)")
    print(f"  Your DST Start (00:00:00): {row['Adjusted_DST_Start_00h00']} ← USE THIS DATE")
    print(f"  Official DST End: {row['Official_DST_End']} (changes at 2:00 AM)")
    print(f"  Your DST End (00:00:00): {row['Adjusted_DST_End_00h00']} ← USE THIS DATE")
    print()

# Save to CSV
df.to_csv('dst_dates_adjusted_for_midnight.csv', index=False)
print("Data saved to 'dst_dates_adjusted_for_midnight.csv'")

# Create a summary table
summary_df = df[['Year', 'Adjusted_DST_Start_00h00', 'Adjusted_DST_End_00h00']].copy()
summary_df.columns = ['Year', 'DST_Start_Date_for_00h00', 'DST_End_Date_for_00h00']

print("\n" + "=" * 60)
print("FINAL DATES TO USE FOR YOUR 00:00:00 CALCULATIONS:")
print("=" * 60)
print(summary_df.to_string(index=False))

DST DATES ADJUSTED FOR 00:00:00 TIMESTAMP CALCULATIONS
LOGIC:
- Official DST starts at 2:00 AM, so 00:00:00 on that day is still Standard Time
- For 00:00:00 calculations, use the NEXT DAY as your DST start date
- DST ends at 2:00 AM, so 00:00:00 on end day is still Daylight Time
- For 00:00:00 calculations, use the SAME DAY as your DST end date

1997:
  Official DST Start: April 6 (changes at 2:00 AM)
  Your DST Start (00:00:00): April 07 ← USE THIS DATE
  Official DST End: October 26 (changes at 2:00 AM)
  Your DST End (00:00:00): October 26 ← USE THIS DATE

1998:
  Official DST Start: April 5 (changes at 2:00 AM)
  Your DST Start (00:00:00): April 06 ← USE THIS DATE
  Official DST End: October 25 (changes at 2:00 AM)
  Your DST End (00:00:00): October 25 ← USE THIS DATE

1999:
  Official DST Start: April 4 (changes at 2:00 AM)
  Your DST Start (00:00:00): April 05 ← USE THIS DATE
  Official DST End: October 31 (changes at 2:00 AM)
  Your DST End (00:00:00): October 31 ← USE THIS DAT

In [2]:
df

,Year,Official_DST_Start,Adjusted_DST_Start_00h00,Official_DST_End,Adjusted_DST_End_00h00,UTC_Offset_Change
0,1997,April 6,April 07,October 26,October 26,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
1,1998,April 5,April 06,October 25,October 25,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
2,1999,April 4,April 05,October 31,October 31,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
3,2000,April 2,April 03,October 29,October 29,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
4,2001,April 1,April 02,October 28,October 28,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
5,2002,April 7,April 08,October 27,October 27,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
6,2003,April 6,April 07,October 26,October 26,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
7,2004,April 4,April 05,October 31,October 31,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
8,2005,April 3,April 04,October 30,October 30,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"
9,2006,April 2,April 03,October 29,October 29,"UTC-5 → UTC-4 (Start), UTC-4 → UTC-5 (End)"


In [3]:
# 1. Prepare the DST DataFrame (as before)
dst_official = [
    {"Year": 1997, "Official_DST_Start": "1997-04-06", "Official_DST_End": "1997-10-26"},
    {"Year": 1998, "Official_DST_Start": "1998-04-05", "Official_DST_End": "1998-10-25"},
    {"Year": 1999, "Official_DST_Start": "1999-04-04", "Official_DST_End": "1999-10-31"},
    {"Year": 2000, "Official_DST_Start": "2000-04-02", "Official_DST_End": "2000-10-29"},
    {"Year": 2001, "Official_DST_Start": "2001-04-01", "Official_DST_End": "2001-10-28"},
    {"Year": 2002, "Official_DST_Start": "2002-04-07", "Official_DST_End": "2002-10-27"},
    {"Year": 2003, "Official_DST_Start": "2003-04-06", "Official_DST_End": "2003-10-26"},
    {"Year": 2004, "Official_DST_Start": "2004-04-04", "Official_DST_End": "2004-10-31"},
    {"Year": 2005, "Official_DST_Start": "2005-04-03", "Official_DST_End": "2005-10-30"},
    {"Year": 2006, "Official_DST_Start": "2006-04-02", "Official_DST_End": "2006-10-29"},
    {"Year": 2007, "Official_DST_Start": "2007-03-11", "Official_DST_End": "2007-11-04"},
    {"Year": 2008, "Official_DST_Start": "2008-03-09", "Official_DST_End": "2008-11-02"},
    {"Year": 2009, "Official_DST_Start": "2009-03-08", "Official_DST_End": "2009-11-01"},
    {"Year": 2010, "Official_DST_Start": "2010-03-14", "Official_DST_End": "2010-11-07"},
    {"Year": 2011, "Official_DST_Start": "2011-03-13", "Official_DST_End": "2011-11-06"},
    {"Year": 2012, "Official_DST_Start": "2012-03-11", "Official_DST_End": "2012-11-04"},
]
dst = pd.DataFrame(dst_official)
dst["Official_DST_Start"] = pd.to_datetime(dst["Official_DST_Start"]).dt.date
dst["Official_DST_End"]   = pd.to_datetime(dst["Official_DST_End"]).dt.date

In [4]:
dst

,Year,Official_DST_Start,Official_DST_End
0,1997,1997-04-06,1997-10-26
1,1998,1998-04-05,1998-10-25
2,1999,1999-04-04,1999-10-31
3,2000,2000-04-02,2000-10-29
4,2001,2001-04-01,2001-10-28
5,2002,2002-04-07,2002-10-27
6,2003,2003-04-06,2003-10-26
7,2004,2004-04-04,2004-10-31
8,2005,2005-04-03,2005-10-30
9,2006,2006-04-02,2006-10-29


In [7]:
from datetime import date

In [14]:
from datetime import date

def is_in_dst(check_date_str: str) -> bool:
    """
    Return True if the date string (YYYY-MM-DD) falls in any DST interval (start <= date < end).
    Time of check_date is 09:00:00 is taken as default
    """
    # Convert input string to date object
    check_date = datetime.strptime(check_date_str, "%Y-%m-%d").date()

    year = check_date.year
    # Filter to the row matching the year
    row = dst[dst["Year"] == year]
    if row.empty:
        return False

    start = row.iloc[0]["Official_DST_Start"]
    end   = row.iloc[0]["Official_DST_End"]
    return start <= check_date < end  # < is used for start to exclude offial end date

In [15]:
date1 = date(1997, 4, 6)

In [16]:
not dst.loc[dst['Year']==date1.year][(dst['Official_DST_Start'] <= date1) & (date1 < dst['Official_DST_End'])].reset_index().empty

/tmp/ipython-input-987010046.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not dst.loc[dst['Year']==date1.year][(dst['Official_DST_Start'] <= date1) & (date1 < dst['Official_DST_End'])].reset_index().empty


True

In [17]:
import re

input_file_path = '/content/sec_actions_enforcement.json'
output_file_path = '/content/sec_actions_enforcement_with_time.json'

# The regex pattern to find "releasedAt":"YYYY-MM-DD"
pattern = r'("releasedAt":\s*)"(\d{4}-\d{2}-\d{2})"'
pattern_regex = re.compile(pattern)

# The replacement string to add the time component
# Here, we use a single, consistent time and offset
replacement_without_dst = r'\1"\2T09:00:00-05:00"'
replacement_with_dst = r'\1"\2T09:00:00-04:00"'

# Open the input and output files
with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
    for line in infile:
        releaseAt = pattern_regex.search(line)
        if releaseAt:
            releaseAt_date_str = releaseAt.group(2)
            if is_in_dst(releaseAt_date_str):
                updated_line = pattern_regex.sub(replacement_with_dst, line)
            else:
                updated_line = pattern_regex.sub(replacement_without_dst, line)
            outfile.write(updated_line)
        else:
            outfile.write(line)

print(f"File processing complete. The updated data is saved in {output_file_path}.")

File processing complete. The updated data is saved in /content/sec_actions_enforcement_with_time.json.


In [18]:
import os
from google.cloud import bigquery

# After uploading the file via UI, give the path to the file on notebook VM
json_file_path = "/content/sec_actions_enforcement_with_time.json"  # example path in the notebook VM

client = bigquery.Client()  # Uses project associated with the notebook

dataset_id = "sec_enforcement_actions"
table_id = "sec_actions"

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE_DATA,
)

with open(json_file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    job.result()

print(f"Loaded {job.output_rows} rows into {table_id}")

Loaded 2760 rows into sec_actions
